In [1]:
import tensorrt as trt
import torch
from det3d.torchie import Config
from det3d.models import build_detector
from det3d.torchie.trainer import load_checkpoint

config = "/workspace/centerformer/configs/nusc/nuscenes_centerformer_poolformer.py"
cfg = Config.fromfile(config)

cfg.work_dir = "/workspace/centerformer/work_dirs/nuscenes_poolformer"
if 'obj_num' in cfg.test_cfg:
        cfg.model['neck']['obj_num'] = cfg.test_cfg['obj_num']
        print('Use center number {} in inference'.format(cfg.model['neck']['obj_num']))
if 'score_threshold' in cfg.test_cfg:
    cfg.model['neck']['score_threshold'] = cfg.test_cfg['score_threshold']
    print('Use heatmap score threshold {} in inference'.format(cfg.model['neck']['score_threshold']))


Deformable Convolution not built!
Deformable Convolution not built!
Use center number 500 in inference
Use heatmap score threshold 0.03 in inference


/data/centerformer/det3d/core/bbox/geometry.py:149: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(nopython=False)
/data/centerformer/det3d/core/bbox/geometry.py:162: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(nopython=False)
/data/centerformer/det3d/core/bbox/geometry.py:280: NumbaDeprecationWarning: The 'nopython' keyword argument was

In [2]:
checkpoint_path = "/workspace/centerformer/work_dirs/nuscenes_poolformer/poolformer.pth"

model = build_detector(cfg.model, train_cfg=None, test_cfg=cfg.test_cfg)
checkpoint = load_checkpoint(model, checkpoint_path, map_location="cpu")
model.cuda()
model.eval();

Use HM Bias:  -2.19


In [3]:
from det3d.datasets import build_dataloader, build_dataset
import os
cwd = os.getcwd()
os.chdir("/workspace/centerformer")

print("Use Val Set")
dataset = build_dataset(cfg.data.val)

data_loader = build_dataloader(
    dataset,
    batch_size=cfg.data.samples_per_gpu,
    workers_per_gpu=cfg.data.workers_per_gpu,
    dist=False,
    shuffle=False,
)

os.chdir(cwd)

no apex
No Tensorflow
Use Val Set
use gt label assigning kernel size  1
10


In [4]:
def load_engine(engine_filepath):
    TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
    with open(engine_filepath, "rb") as f, trt.Runtime(TRT_LOGGER) as runtime:
        engine = runtime.deserialize_cuda_engine(f.read())
    return engine

centerFinder_engine_path = 'findCenter.trt'
cf_engine = load_engine(centerFinder_engine_path)
cf_context = cf_engine.create_execution_context()

[12/13/2023-16:47:25] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage. See `CUDA_MODULE_LOADING` in https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#env-vars


In [5]:
def run_trt_engine(context, engine, tensors):
    bindings = [None]*engine.num_bindings
    for idx, binding in enumerate(engine):
        tensor_name = engine.get_tensor_name(idx)
        if engine.get_tensor_mode(binding)==trt.TensorIOMode.INPUT:
            bindings[idx] = tensors['inputs'][tensor_name].data_ptr()
            if context.get_tensor_shape(tensor_name):
                context.set_input_shape(tensor_name, tensors['inputs'][tensor_name].shape)
        else:
            bindings[idx] = tensors['outputs'][tensor_name].data_ptr()
    context.execute_v2(bindings=bindings)


In [6]:
from det3d.torchie.apis.train import example_to_device
from det3d import torchie

prog_bar = torchie.ProgressBar(len(data_loader.dataset) // 1)

detections = {}

for i, data_batch in enumerate(data_loader):
    device = torch.device("cuda")
    example = example_to_device(data_batch, device, non_blocking=False)
    with torch.no_grad():
        example = example_to_device(data_batch, device, non_blocking=False)
        del data_batch
        with torch.no_grad():
            # outputs = model(example, return_loss=False)
            reader_output = model.reader(example['points'])    
            voxels, coors, shape = reader_output
            x, _ = model.backbone(voxels, coors, len(example['points']), shape)
            
            # Neck
            # ct_feat, center_pos_embedding, out_scores, out_labels, out_orders, out_masks = model.neck.find_centers(x)
            ct_feat = torch.zeros((1, 3000, 256), dtype=torch.float32, device=x.device)
            center_pos_embedding = torch.zeros((1, 3000, 256), dtype=torch.float32, device=x.device)
            out_scores = torch.zeros((6, 1, 500), dtype=torch.float32, device=x.device)
            out_labels = torch.zeros((6, 1, 500), dtype=torch.int32, device=x.device)
            out_orders = torch.zeros((6, 1, 500), dtype=torch.int32, device=x.device)
            out_masks = torch.zeros((6, 1, 500), dtype=torch.bool, device=x.device)
            IO_tensors = {
                "inputs" :
                {'input_tensor': x},
                "outputs" :
                {'ct_feat': ct_feat, 'center_pos_embedding': center_pos_embedding,
                'out_scores': out_scores, 'out_labels': out_labels,
                'out_orders': out_orders, 'out_masks': out_masks}
            }
            run_trt_engine(cf_context, cf_engine, IO_tensors)
            
            ct_feat = model.neck.poolformer_forward(ct_feat, center_pos_embedding)
            
            out_dict_list = []
            
            for idx in range(len(cfg.tasks)):
                out_dict = {}
                out_dict.update(
                    {
                        "scores": out_scores[idx],
                        "labels": out_labels[idx],
                        "order": out_orders[idx],
                        "mask": out_masks[idx],
                        "ct_feat": ct_feat[:, :, idx * cfg.test_cfg['obj_num'] : (idx+1) * cfg.test_cfg['obj_num']],
                    }
                )
                out_dict_list.append(out_dict)
                
            out_dict_list2 = out_dict_lists = model.neck.forward(x)
            
            print(out_dict_list[0]['scores'])
            print(out_dict_list2[0]['scores'])
            assert False
                
            preds = model.bbox_head(out_dict_list)
            outputs = model.bbox_head.predict(example, preds, model.test_cfg)
            
        for output in outputs:
            token = output["metadata"]["token"]
            for k, v in output.items():
                if k not in [
                    "metadata",
                ]:
                    output[k] = v.to(device="cpu")
            detections.update(
                {token: output,}
            )
            
            prog_bar.update()

No APEX!
[                              ] 0/6019, elapsed: 0s, ETA:

/root/anaconda3/envs/centerformer/lib/python3.9/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


tensor([[0.8610, 0.8600, 0.8393, 0.8083, 0.8044, 0.7641, 0.7413, 0.7288, 0.7048,
         0.6847, 0.6831, 0.6785, 0.6444, 0.6397, 0.6353, 0.6312, 0.6230, 0.5880,
         0.5813, 0.5762, 0.5750, 0.5687, 0.5622, 0.5600, 0.5405, 0.5021, 0.4977,
         0.4964, 0.4836, 0.4738, 0.4396, 0.4358, 0.4336, 0.4208, 0.4134, 0.4042,
         0.4001, 0.3994, 0.3968, 0.3764, 0.3758, 0.3660, 0.3656, 0.3572, 0.3466,
         0.3457, 0.3433, 0.3372, 0.3363, 0.3348, 0.3094, 0.3040, 0.3037, 0.3031,
         0.3031, 0.2965, 0.2842, 0.2781, 0.2743, 0.2723, 0.2705, 0.2680, 0.2657,
         0.2657, 0.2646, 0.2628, 0.2578, 0.2462, 0.2393, 0.2391, 0.2384, 0.2338,
         0.2336, 0.2317, 0.2279, 0.2277, 0.2254, 0.2220, 0.2208, 0.2204, 0.2197,
         0.2185, 0.2176, 0.2067, 0.2008, 0.1964, 0.1943, 0.1932, 0.1913, 0.1866,
         0.1856, 0.1832, 0.1827, 0.1806, 0.1770, 0.1770, 0.1758, 0.1746, 0.1738,
         0.1730, 0.1728, 0.1724, 0.1720, 0.1719, 0.1707, 0.1667, 0.1661, 0.1639,
         0.1632, 0.1630, 0.1

AssertionError: 